In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, UpSampling2D, MaxPooling2D, Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
data_dir = '../../../results'

def read_images_from_dir(directory):
    images = []
    for filename in os.listdir(directory):
        img = cv2.imread(os.path.join(directory, filename))
        if img is not None:
            images.append(img)
    return images

images = np.array(read_images_from_dir(os.path.join(data_dir, 'images')))
masks = np.array(read_images_from_dir(os.path.join(data_dir, 'masks')))

In [ ]:
images = images[:]
masks = masks[:]

masks = (masks / 255).astype(np.uint8)
# masks = np.where(masks == 255, 1, 0)

In [ ]:
train_images, test_images, train_masks, test_masks = train_test_split(images, masks, test_size=0.1, random_state=42)
val_images = test_images
val_masks = test_masks

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(120, 120, 3))
base_model = Model(inputs=base_model.input, outputs=base_model.layers[:-16][-1].output)

In [ ]:
x = Conv2D(32, (3, 3), activation='relu', padding='same')(base_model.output)
x = Conv2D(16, (3, 3), activation='sigmoid', padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer = Adam(learning_rate=0.0001),
                loss='binary_crossentropy',
                metrics=['accuracy'])

model.fit(train_images,
          train_masks,
          validation_data=(val_images, val_masks),
          steps_per_epoch=6,
          epochs=10,
          batch_size=64)

loss, accuracy = model.evaluate(test_images, test_masks)

In [ ]:
print(model.summary())

In [ ]:
predicted_masks = []

for image in test_images[:5]:
    prediction = model.predict(np.array([image]))[0]
    predicted_masks.append(prediction)

In [ ]:
predicted_masks_modified = []

for mask in predicted_masks:
    modified_mask = np.where(mask > 0.5, 255, 0)
    predicted_masks_modified.append(modified_mask)

In [ ]:
def plot_images(ground_truth, predicted):
    num_images = len(predicted)
    plt.figure(figsize=(10, 5 * num_images))
    for i in range(num_images):
        plt.subplot(num_images, 2, 2*i + 1)
        plt.imshow(predicted[i], cmap='gray')
        plt.title('Predicted')
        plt.axis('off')

        plt.subplot(num_images, 2, 2*i + 2)
        plt.imshow(ground_truth[i] * 255, cmap='gray')
        plt.title('Ground Truth')
        plt.axis('off')

plot_images(val_masks, predicted_masks_modified)
plt.show()